In [7]:
import pandas as pd
import numpy as np
import wbgapi as wb
from pandas_profiling import ProfileReport

In [83]:
# Pour ces deux bases de données on a pas des données annuels ?? 

#vulnérabilité climatique
#df_clim = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/donnees-indicateur-de-vulnerabilite-physique-au-changement-climatique (1).xlsx')
df_clim = pd.read_excel('../clim.xlsx', engine='openpyxl')

#fractionalisation sociale
#df_frac = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/indicateurs-de-fragmentation-ethnolinguistique.xls')
df_frac = pd.read_excel('../indicateurs-de-fragmentation-ethnolinguistique.xls')[['country_name','fearon']]

In [46]:
#indice de capital humain
df = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/data-series-retrospectives-de-l-indice-hai.xlsx').drop('iso_3', axis = 1)

df_hai = pd.wide_to_long(df_hai, ['HAI'], i= 'country', j = 'year').reset_index()# de 1990 à 2014
df_hai

,country,year,HAI
0,Afghanistan,1990,15.131342
1,Algeria,1990,66.503784
2,Angola,1990,13.832288
3,Antigua and Barbuda,1990,91.676514
4,Argentina,1990,87.962227
...,...,...,...
3620,Venezuela (Bolivarian Republic of),2014,95.271568
3621,Viet Nam,2014,93.750504
3622,Yemen,2014,62.100121
3623,Zambia,2014,37.954849


In [84]:
# Données sur l'instabilité politique

#df_insta_pol = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/p5v2018.xls')
df_insta_pol = pd.read_excel('../p5v2018.xls')


df_insta_pol = df_insta_pol[df_insta_pol['year']>= 1989][['country','year','polity2']] #de 1989 à 2018

In [85]:
#vulnérabilité économique structurelle
#df1 = pd.read_excel(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/indicateur-de-vulnerabilite.xlsx')
df_eco = pd.read_excel('../indicateur-de-vulnerabilite.xlsx')[['country','year','EVI']]


df_eco = df_eco[df_eco['year']>=1989]

In [8]:
# Création d'une deuxième base comportant des variables par pays et année que l'on utilisera dans la partie modélisation

#Population totale, PIB par habitant (en $), Indice GINI, Croissance annuelle du PIB, taux de chomage, mortalité infantile, indice de la valeur des exportations, IDE
data_wb= wb.data.DataFrame(['SP.POP.TOTL','NY.GDP.PCAP.KD','NY.GDP.MKTP.KD.ZG','SH.DTH.IMRT','SI.POV.GINI', 'SL.UEM.TOTL.NE.ZS','TX.VAL.MRCH.XD.WD', 'BM.KLT.DINV.CD.WD'], labels =True, time = range(1989, 2020), columns = 'series').reset_index().rename(columns = {'Time': 'year', 'SP.POP.TOTL' : 'Population', 'NY.GDP.PCAP.KD' : 'PIB/hab', 'NY.GDP.MKTP.KD.ZG' : 'CroissancePIB', 'SH.DTH.IMRT' : 'nb d enfants morts', 'SI.POV.GINI' : 'indice de Gini', 'SL.UEM.TOTL.NE.ZS ' : 'taux de chômage', 'TX.VAL.MRCH.XD.WD' : 'indice de la valeur des exportations', 'BM.KLT.DINV.CD.WD' : 'IDE'}).drop(columns = ['time','economy'])

#on regarde le pourcentage de valeurs manquantes
percent= 100*(len(data_wb.loc[:,data_wb.isnull().sum(axis=0)>=1 ].index) / len(data_wb.index))

In [87]:
data_wb.rename(columns = {'Country':'country'}, inplace = True)
data_wb.year = data_wb.year.astype(int)

In [88]:
# On s'assure que les pays sur lesquelles on va faire la jointure soient bien les mêmes dans les df
cntry_list = df_eco.country.unique()

data_wb = data_wb[data_wb.country.isin(cntry_list)][data_wb.year <= 2014]
df_hai = df_hai[df_hai.country.isin(cntry_list)]  #j'ai rajouté ça
data_eco = df_eco[df_eco.country != 'Korea, Dem. People’s Rep.']
data_pol = df_insta_pol[df_insta_pol.country.isin(cntry_list)]

data_wb = data_wb[data_wb.country.isin(data_pol.country.unique())]
data_eco = data_eco[data_eco.country.isin(data_pol.country.unique())]

<ipython-input-88-0b4c94137593>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_wb = data_wb[data_wb.country.isin(cntry_list)][data_wb.year <= 2018]


In [89]:
df_merge1 = pd.merge(data_wb, data_eco, on=['country', 'year'])
df_merge2 = pd.merge(test1, data_pol, on=['country', 'year'])

In [90]:
df_merge2

,country,year,Croissance PIB,PIB/hab,Population,EVI,polity2
0,Zimbabwe,2018,4.824211,1509.523735,14438812.0,57.888145,4.0
1,Zimbabwe,2017,4.709492,1479.233468,14236599.0,57.711140,4.0
2,Zimbabwe,2016,0.755869,1433.608776,14030338.0,57.590748,4.0
3,Zimbabwe,2015,1.779873,1445.069702,13814642.0,58.380211,4.0
4,Zimbabwe,2014,2.376929,1443.617777,13586710.0,58.104874,4.0
...,...,...,...,...,...,...,...
3098,Afghanistan,1993,NaN,NaN,15816601.0,30.988297,0.0
3099,Afghanistan,1992,NaN,NaN,14485543.0,30.419651,0.0
3100,Afghanistan,1991,NaN,NaN,13299016.0,31.579372,-8.0
3101,Afghanistan,1990,NaN,NaN,12412311.0,31.802521,-8.0


In [121]:
# Provisoirement l'indicateur utilisé est la proportion qu'occupe un pays donné dans le nombre total de mort au cours d'une année donnée

df_ucdp = pd.read_csv('../UCDP.csv')

def ratio_best(country,y):
    # Nombre d'années que le conflit est en activité durant jusqu'à l'année 
        
    return df_ucdp[df_ucdp['year'] == y][df_ucdp['country'] == country]['best'].sum()/df_ucdp[df_ucdp['year'] == y]['best'].sum()

df_indicator = pd.DataFrame({'country':[],
        'year': [],
        'ratio':[]})

for y in df_ucdp['year'].unique():
    for cntry in df_ucdp[df_ucdp['year'] == y]['country'].unique():

        df_indicator = df_indicator.append({'country':cntry,'year':y,'ratio':ratio_best(cntry,y)},ignore_index=True)

UsageError: Line magic function `%%capture` not found.


In [113]:
# On s'assure que les pays sur lesquelles on va faire la jointure soient bien les mêmes dans les df

cntry_list = df_merge2.country.unique()
data_indicator = df_indicator[df_indicator.country.isin(cntry_list)][df_indicator.year <= 2014]
df_merge2 = df_merge2[df_merge2.country.isin(data_indicator.country.unique())]

<ipython-input-113-321faed46129>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_indicator = df_indicator[df_indicator.country.isin(cntry_list)][df_indicator.year <= 2018]


In [116]:
df_merge3 = pd.merge(data_indicator, df_merge2, on=['country', 'year'])

In [117]:
df_merge3

,country,year,ratio,Croissance PIB,PIB/hab,Population,EVI,polity2
0,Afghanistan,2017.0,0.199614,2.647003,575.707053,36296111.0,38.267334,-1.0
1,Algeria,2017.0,0.000777,1.300000,4192.335033,41389174.0,14.174278,2.0
2,Angola,2017.0,0.000262,-0.147213,3790.791565,29816769.0,39.088776,-2.0
3,Azerbaijan,2017.0,0.000363,0.200000,5229.068864,9854033.0,30.499851,-7.0
4,Bangladesh,2017.0,0.000485,7.284174,1403.861692,159685421.0,24.695305,1.0
...,...,...,...,...,...,...,...,...
1235,Tanzania,2001.0,0.000994,6.070808,539.527905,34385849.0,41.860069,-1.0
1236,Thailand,2001.0,0.000672,3.444249,3605.368235,63539190.0,25.436676,9.0
1237,Turkey,2001.0,0.002876,-5.750007,6075.995627,64192243.0,13.717976,7.0
1238,Uganda,2001.0,0.005698,5.183661,524.790767,24388974.0,35.405231,-4.0


In [120]:
# On exporte les données rassemblées 

df_merge3.to_csv('../df_regression.csv', index=False)
#df_merge3.to_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/df_regression.csv', index=False)